# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os

# Set your preferred name
USER = "Jinglin"
# Set the data and output directories
DATA_ROOT = '../../DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Multiple Chemical Sensitivity'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [5]:
trait_subdir = "TCGA_Thyroid_Cancer_(THCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.THCA.sampleMap_THCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.THCA.sampleMap_HiSeqV2_PANCAN.gz')

In [6]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [7]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [8]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'additional_surgery_locoregional_procedure',
 'age_at_initial_pathologic_diagnosis',
 'bcr_followup_barcode']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [9]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'braf_gene_genotyping_outcome_lab_results_text', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathol

In [12]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis',
                      'days_to_birth']
candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [13]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [74, 65, 55, 57, 33],
 'days_to_birth': [-27348, -24102, -20361, -21008, -12295]}

In [16]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [17]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}

In [18]:
gender_col = 'gender'

In [19]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [20]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [21]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Thyroid Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-4C-A93U-01,1,74,0,-0.767592,-1.742926,-0.531035,-0.067972,0.265922,-0.13531,0.440006,...,-0.748878,-1.018517,0.205567,-1.28139,0.395523,1.735414,0.945273,-0.186695,4.024767,-0.273612
TCGA-BJ-A0YZ-01,1,65,1,1.766308,0.273374,-0.531035,0.790028,1.209322,0.01229,0.704606,...,0.125322,1.568883,0.626367,2.38101,0.275823,1.820014,2.140073,1.380405,1.257367,-0.174312
TCGA-BJ-A0Z0-01,1,55,1,0.986808,-2.165726,-0.050935,0.051428,0.979522,0.48229,0.090106,...,-0.748878,-0.747417,-0.931133,-1.28139,-0.222077,0.293414,0.725173,-0.254095,0.530667,-0.430812
TCGA-BJ-A0Z2-01,1,57,1,0.767208,-1.623626,-0.531035,0.182228,0.541622,0.65889,0.647206,...,-0.748878,-1.587117,-1.292433,-1.28139,-0.602177,0.134114,0.894473,-0.380395,-2.502833,0.360588
TCGA-BJ-A0Z3-01,1,33,0,-0.287292,-1.766626,-0.531035,0.059628,0.920522,0.21849,0.336106,...,-0.413778,1.111583,1.158867,-1.28139,0.003723,1.240514,0.924373,-0.143495,2.613367,-0.030012


In [22]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 572 samples.


In [23]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Thyroid Cancer', the least common label is '0' with 59 occurrences. This represents 10.31% of the dataset.
The distribution of the feature 'Thyroid Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 34.0
  50% (Median): 46.0
  75%: 58.0
Min: 15
Max: 89
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 157 occurrences. This represents 27.45% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [24]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [25]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Jinglin/Thyroid-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [5]:
dataset = 'GEO'
trait_subdir = "Multiple-Chemical-Sensitivity"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE182797', 'GSE182798', 'GSE182503']

Repeat the below steps for all the accession numbers

In [59]:
cohort = accession_num = "GSE182798"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('../../DATA/GEO/Multiple-Chemical-Sensitivity/GSE182798/GSE182798_family.soft.gz',
 '../../DATA/GEO/Multiple-Chemical-Sensitivity/GSE182798/GSE182798_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [60]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Transcriptomic profiling of adult-onset asthma related to damp and moldy buildings and idiopathic environmental intolerance"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"


In [61]:
clinical_data.head()

,!Sample_geo_accession,GSM5530417,GSM5530418,GSM5530419,GSM5530420,GSM5530421,GSM5530422,GSM5530423,GSM5530424,GSM5530425,...,GSM5537227,GSM5537228,GSM5537229,GSM5537230,GSM5537231,GSM5537232,GSM5537233,GSM5537234,GSM5537235,GSM5537236
0,!Sample_characteristics_ch1,diagnosis: adult-onset asthma,diagnosis: adult-onset asthma,diagnosis: IEI,diagnosis: healthy,diagnosis: healthy,diagnosis: adult-onset asthma,diagnosis: healthy,diagnosis: adult-onset asthma,diagnosis: adult-onset asthma,...,diagnosis: IEI,diagnosis: healthy,diagnosis: adult-onset asthma,diagnosis: healthy,diagnosis: IEI,diagnosis: healthy,diagnosis: adult-onset asthma,diagnosis: adult-onset asthma,diagnosis: adult-onset asthma,diagnosis: healthy
1,!Sample_characteristics_ch1,gender: Female,gender: Female,gender: Female,gender: Female,gender: Female,gender: Female,gender: Female,gender: Female,gender: Female,...,gender: Female,gender: Female,gender: Female,gender: Female,gender: Female,gender: Female,gender: Female,gender: Female,gender: Female,gender: Female
2,!Sample_characteristics_ch1,age: 33.42,age: 46.08,age: 45.58,age: 28,age: 25.75,age: 59.83,age: 41.17,age: 47.58,age: 50.75,...,age: 50.33,age: 37.83,age: 44.25,age: 58.83,age: 48.25,age: 43.08,age: 41.17,age: 51.75,age: 53.58,age: 41.5
3,!Sample_characteristics_ch1,cell type: PBMC,cell type: PBMC,cell type: PBMC,cell type: PBMC,cell type: PBMC,cell type: PBMC,cell type: PBMC,cell type: PBMC,cell type: PBMC,...,tissue: Nasal biopsy,tissue: Nasal biopsy,tissue: Nasal biopsy,tissue: Nasal biopsy,tissue: Nasal biopsy,tissue: Nasal biopsy,tissue: Nasal biopsy,tissue: Nasal biopsy,tissue: Nasal biopsy,tissue: Nasal biopsy
4,!Sample_characteristics_ch1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,subject: 7,subject: 639,subject: 33,subject: 625,subject: 32,subject: 623,subject: 62,subject: 80,subject: 78,subject: 620


In [62]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['diagnosis: adult-onset asthma', 'diagnosis: IEI', 'diagnosis: healthy'],
 1: ['gender: Female'],
 2: ['age: 33.42',
  'age: 46.08',
  'age: 45.58',
  'age: 28',
  'age: 25.75',
  'age: 59.83',
  'age: 41.17',
  'age: 47.58',
  'age: 50.75',
  'age: 42.58',
  'age: 52.75',
  'age: 51.75',
  'age: 18.42',
  'age: 47',
  'age: 38.33',
  'age: 58.58',
  'age: 56.17',
  'age: 40.67',
  'age: 47.5',
  'age: 54.67',
  'age: 48.83',
  'age: 64.67',
  'age: 54.83',
  'age: 57.67',
  'age: 39.17',
  'age: 38.08',
  'age: 28.42',
  'age: 40.75',
  'age: 43.17',
  'age: 43.08'],
 3: ['cell type: PBMC', 'tissue: Nasal biopsy'],
 4: [nan,
  'subject: 605',
  'subject: 611',
  'subject: 621',
  'subject: 35',
  'subject: 11',
  'subject: 1',
  'subject: 601',
  'subject: 54',
  'subject: 68_A',
  'subject: 55',
  'subject: 44',
  'subject: 603_A',
  'subject: 63',
  'subject: 39',
  'subject: 13',
  'subject: 3',
  'subject: 619',
  'subject: 58',
  'subject: 79',
  'subject: 77',
  'subject: 4

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [65]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.
   NOTE: Function "convert_trait" should return 1 or 0.
Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Multiple Chemical Sensitivity\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Multiple Chemical Sensitivity\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [66]:
age_row = gender_row = None
convert_age = convert_gender = None

In [67]:
is_gene_availabe = True
trait_row = 0
age_row = 2
gender_row = None

trait_type = 'binary'

In [68]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [69]:
def convert_trait(trait_value):
    """
    Convert the trait value to a binary data type for Multiple Chemical Sensitivity.
    Returns 1 if the trait is 'diagnosis: IEI', 0 for other diagnoses, and None for unknown values.
    """
    if 'IEI' in trait_value:
        return 1
    elif 'healthy' in trait_value:
        return 0
    else:
        return None

def convert_age(age_value):
    """
    Convert the age value to a continuous data type.
    Returns the numerical age or None for unknown values.
    """
    try:
        age = float(age_value.split(': ')[1])
        return age
    except (ValueError, IndexError):
        return None

def convert_gender(gender_value):
    """
    Convert the gender value to a binary data type.
    Returns 1 for 'gender: Female', and None for unknown values.
    """
    if 'Female' in gender_value:
        return 1
    else:
        return None

In [70]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM5530417,GSM5530418,GSM5530419,GSM5530420,GSM5530421,GSM5530422,GSM5530423,GSM5530424,GSM5530425,GSM5530426,...,GSM5537227,GSM5537228,GSM5537229,GSM5537230,GSM5537231,GSM5537232,GSM5537233,GSM5537234,GSM5537235,GSM5537236
Multiple Chemical Sensitivity,NaN,NaN,1.00,0.0,0.00,NaN,0.00,NaN,NaN,0.00,...,1.00,0.00,NaN,0.00,1.00,0.00,NaN,NaN,NaN,0.0
Age,33.42,46.08,45.58,28.0,25.75,59.83,41.17,47.58,50.75,42.58,...,50.33,37.83,44.25,58.83,48.25,43.08,41.17,51.75,53.58,41.5


### Genetic data preprocessing and final filtering

In [71]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM5530417,GSM5530418,GSM5530419,GSM5530420,GSM5530421,GSM5530422,GSM5530423,GSM5530424,GSM5530425,GSM5530426,...,GSM5537227,GSM5537228,GSM5537229,GSM5537230,GSM5537231,GSM5537232,GSM5537233,GSM5537234,GSM5537235,GSM5537236
ID,,,,,,,,,,,,,,,,,,,,,
A_19_P00315452,6.98856,7.06609,6.54699,7.25514,7.45419,7.72615,7.89290,7.20461,6.96327,7.01397,...,5.89786,5.94732,6.00343,6.42312,5.87709,6.07091,5.88997,6.06217,5.87444,6.31630
A_19_P00315492,5.22609,5.27459,5.38136,5.17546,5.18752,5.34490,5.13498,5.30717,5.17996,5.19062,...,5.19988,5.11501,5.16016,5.16470,5.08577,5.07849,5.18470,5.11267,5.25578,5.09046
A_19_P00315493,6.67292,7.17868,7.71182,6.64976,7.16333,6.74629,6.96400,7.81434,7.64035,7.23596,...,7.97988,7.87559,7.76055,7.82140,8.04265,7.91478,7.60659,7.57412,7.76431,7.89814
A_19_P00315506,8.97925,9.05869,9.27026,8.95605,8.89901,8.50964,8.55109,8.92043,8.57666,8.41800,...,8.72588,8.81576,8.94775,8.99971,9.18310,8.71062,8.97180,8.99259,9.07988,8.83203
A_19_P00315529,6.79033,6.84270,6.79024,6.80470,6.64201,6.58049,6.51791,6.69308,6.40250,6.56796,...,5.46832,5.43659,5.34555,5.37979,5.51219,5.57225,5.42150,5.13288,5.38903,5.47817


In [72]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['A_19_P00315452',
 'A_19_P00315492',
 'A_19_P00315493',
 'A_19_P00315506',
 'A_19_P00315529',
 'A_19_P00315551',
 'A_19_P00315581',
 'A_19_P00315584',
 'A_19_P00315593',
 'A_19_P00315603',
 'A_19_P00315627',
 'A_19_P00315649',
 'A_19_P00315668',
 'A_19_P00315691',
 'A_19_P00315705',
 'A_19_P00315716',
 'A_19_P00315753',
 'A_19_P00315764',
 'A_19_P00315773',
 'A_19_P00315780']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [73]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['A_19_P00315452', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315506', 'A_19_P00315529', 'A_19_P00315551', 'A_19_P00315581', 'A_19_P00315584', 'A_19_P00315593', 'A_19_P00315603', 'A_19_P00315627', 'A_19_P00315649', 'A_19_P00315668', 'A_19_P00315691', 'A_19_P00315705', 'A_19_P00315716', 'A_19_P00315753', 'A_19_P00315764', 'A_19_P00315773', 'A_19_P00315780']\n"

If not required, jump directly to the gene normalization step

In [74]:
requires_gene_mapping = True

In [75]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533]', 'BCL6 corepressor [So

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [76]:
# if requires_gene_mapping:
f'''
As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
probe_name_key = key_name1
gene_name_key = key_name2

Gene annotation dictionary:
{gene_annotation_summary}
'''

"\nAs a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:\nprobe_name_key = key_name1\ngene_name_key = key_name2\n\nGene annotation dictionary:\n{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_00110

In [77]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'GENE_SYMBOL'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [78]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

37 input query terms found dup hits:	[('AADACL2-AS1', 3), ('AADACP1', 2), ('ACTA2-AS1', 2), ('ACTG1P20', 2), ('ACTG1P4', 2), ('ACTR3BP5',
37 input query terms found no hit:	['AAED1', 'AARS', 'ACN9', 'ACPP', 'ACPT', 'ACRC', 'ACTN1-AS1', 'ADCK3', 'ADCK4', 'ADPRHL2', 'ADRBK1'
23 input query terms found dup hits:	[('ATP13A5-AS1', 2), ('ATP2B2-IT2', 2), ('ATP6AP1L', 2), ('ATP8B5P', 2), ('B3GALT5-AS1', 2), ('B4GAL
237 input query terms found no hit:	['ATP5A1', 'ATP5B', 'ATP5C1', 'ATP5D', 'ATP5E', 'ATP5F1', 'ATP5G1', 'ATP5G2', 'ATP5G3', 'ATP5H', 'AT
13 input query terms found dup hits:	[('CCDC162P', 2), ('CCNT2-AS1', 2), ('CD99P1', 3), ('CEBPB-AS1', 2), ('CECR7', 2), ('CES1P2', 2), ('
40 input query terms found no hit:	['CCDC129', 'CCDC130', 'CCDC132', 'CCDC147-AS1', 'CCDC151', 'CCDC153', 'CCDC173', 'CCDC176', 'CCDC23
39 input query terms found dup hits:	[('CSAG4', 3), ('CTAGE10P', 2), ('CTAGE11P', 2), ('CTAGE7P', 2), ('CTSLP2', 2), ('CTSLP8', 2), ('CXC
43 input query terms found no hit:	['CS

In [79]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [80]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 87 samples.


In [81]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Multiple Chemical Sensitivity', the least common label is '1.0' with 30 occurrences. This represents 34.48% of the dataset.
The distribution of the feature 'Multiple Chemical Sensitivity' in this dataset is fine.

Quartiles for 'Age':
  25%: 38.205
  50% (Median): 43.08
  75%: 51.75
Min: 18.42
Max: 60.17
The distribution of the feature 'Age' in this dataset is fine.



False

In [82]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [83]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [4]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
2,GSE182503,True,True,False,True,False,102,
1,GSE182798,True,True,False,True,False,87,
0,GSE182797,True,True,False,True,False,35,


In [5]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

'GSE182503'

In [6]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
2,GSE182503,True,True,False,True,False,102,
1,GSE182798,True,True,False,True,False,87,
0,GSE182797,True,True,False,True,False,35,


In [7]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

,Multiple Chemical Sensitivity,Age,A1BG,A1BG-AS1,A2M,A2M-AS1,A4GALT,AAAS,AACS,AADACL2-AS1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.0,33.42,9.58489,6.85151,5.35128,7.373065,6.48037,7.48055,6.054405,5.24763,...,6.93642,6.54968,6.570575,5.52895,9.11228,7.41289,7.053450,9.89924,9.82874,8.529810
1,0.0,46.08,9.65673,6.74284,5.17157,7.897180,6.38845,7.26270,6.050235,5.16678,...,6.82689,6.56084,6.206125,5.44427,9.29162,7.51223,6.758815,9.54241,9.92456,8.431920
2,1.0,45.58,9.55825,6.47113,5.31526,8.515230,6.41518,7.27516,5.995330,5.29596,...,6.72080,6.32144,6.096655,5.55285,8.82925,7.55573,6.817940,9.96698,10.10630,8.269220
3,0.0,28.00,9.71055,6.90045,5.23478,7.498270,6.46742,7.37446,6.082195,5.19923,...,6.77188,6.52291,6.201890,5.60998,9.03057,7.34095,6.787210,9.65593,9.83997,8.400015
4,0.0,25.75,9.22204,6.75952,5.37102,7.955740,6.49131,7.34519,6.067165,5.19351,...,6.91466,6.55797,6.434910,5.55869,8.87090,7.56096,6.933440,10.03720,9.78509,8.247640


In [8]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [9]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [10]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [11]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 50.00% ± 19.49%


In [13]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 90.00% ± 4.47%


In [14]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 4. Discussion and report

In [15]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0657
Found 74 genes with non-zero coefficients associated with the trait 'Multiple Chemical Sensitivity' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
       Variable  Coefficient  Absolute Coefficient
         PTGER3    -6.129210              6.129210
LIMS3-LOC440895     4.982596              4.982596
         ICOSLG    -3.428709              3.428709
           MAP2    -3.126846              3.126846
          TUSC8    -3.107201              3.107201
        CCDC124     3.072731              3.072731
        COL18A1    -2.641617              2.641617
          FKBPL    -2.622397              2.622397
         SLC5A2    -2.618780              2.618780
          YPEL4     2.516959              2.516959
         HIVEP3     2.327110              2.327110
          KAAG1    -2.293216              2.293216
        SLC2A12     2.239383              2.23938

/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [16]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0657
Found 0 significant genes associated with the trait 'Multiple Chemical Sensitivity' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
